Amirreza Hosseini </br> 
HW3-Q2

کتاب خانه‌های مورد نظر را وارد می‌کنیم<br>
این کتابخانه ها عبارتند از : 


*   numpy
*   matplotlib
*   `torch`

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
from torch.utils.data import DataLoader

In [37]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
       # transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
      #  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
       # transforms.CenterCrop(224),
        transforms.ToTensor()
       # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



#download mnist dataset 
data_dir = 'data/mnist'
image_datasets = {x: datasets.MNIST(data_dir, train=x=='train', download=True, transform=data_transforms[x])
                  
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


# ####

<div dir=rtl>
در این قسمت دیتاست گفته شده را دانلود کردیم(با استفاده از خود کتابخانه پایتورچ) و برای اینکه فرآیند خواندن دیتاست سریع تر و بهینه تر باشد از 
DataLoader
استفاده کردیم.
<br>

همچنین با استفاده از data transform داده‌ها را تبدیل به اندازه مورد نظر تبدیل می‌کنیم.

In [36]:

#switch to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)



cpu


**Train Model**

In [38]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

we use **vgg16** to reduce the time and calculation.

In [39]:
class MyModel(nn.Module):
    
    def __init__(self):
        super(MyModel, self).__init__()
        #self.pre_layar = nn.Linear(in_features=1, out_features=3)
        self.conv2d=nn.Conv2d(1, 3, 3,padding=1)
       
        self.backbone = models.vgg16(pretrained=True)
        #self.backbone.modules[6] = nn.Linear(in_features=4096, out_features=2)
        #self.fv1 = nn.Linear(in_features=2, out_features=2)
        self.fv2 = nn.Linear(in_features=1000, out_features=10)
        
    
    def forward(self, x):
       # print(x.size())
        x=self.conv2d(x)
        #print(x.size())
       # x = self.pre_layar(x)
        x = self.backbone(x)
        #print(x.size())
       # x = self.fv1(x)
        x = self.fv2(x)
        #print(x.size())
       
        return x

<div dir=rtl>
در کلاسی که تعریف می‌کنیم یک لایه قبل از شبکه
vgg
تعریف می‌کنیم تا بتوانیم کانال‌های رنگی سیاه سفید را به رنگی تبدیل کنیم زیرا میدانیم
vgg
تنها با عکس‌های رنگی کار می‌کند پس مجبوریم سایز ورودی را تغییر دهیم
<div>
<br>
optimize function=SGD

**first scenario** </br>
finetune: learning the whole model <br>
(requires_grad) default value is TRUE

In [5]:
#give the data to the model
model = MyModel()
model.to(device)


#model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

**train first scenario**

In [ ]:
model_conv = train_model(model, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=2)

**Second scenario** <br>
feature extraction: learning only the new layars and freeze others

In [31]:
class MyModel(nn.Module):
    
    def __init__(self):
        super(MyModel, self).__init__()
        #self.pre_layar = nn.Linear(in_features=1, out_features=3)
        self.conv2d=nn.Conv2d(1, 3, 3,padding=1)
       
        self.backbone = models.vgg16(pretrained=True)
        #self.backbone.modules[6] = nn.Linear(in_features=4096, out_features=2)
        #self.fv1 = nn.Linear(in_features=2, out_features=2)
        self.fv2 = nn.Linear(in_features=1000, out_features=10)

        for param in self.backbone.parameters():
            param.requires_grad = False
    
    def forward(self, x):
       # print(x.size())
        x=self.conv2d(x)
        #print(x.size())
       # x = self.pre_layar(x)
        x = self.backbone(x)
        #print(x.size())
       # x = self.fv1(x)
        x = self.fv2(x)
        #print(x.size())
       
        return x

        
    

<div dir=rtl>
در سناریو دوم نیز تمام پارامتر‌های مدل اصلی را فیریز می‌کنیم و فقط لایه‌های اول و آخر را آپدیت می‌کنیم برای این کار از امکان
param.requires_grad= False
استفاده می‌کنیم
<br>
<div>
optimize function=SGD

In [ ]:
#give the data to the model
model = MyModel()
model.to(device)


#model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
#optimizer_conv = optim.Adam(model.fv1.parameters(), lr=0.001)
#optimizer_conv = optim.Adam(model.fv2.parameters(), lr=0.001)
optimizer_conv = optim.SGD(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

**train the second scenario**


In [ ]:
model_conv = train_model(model, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=2)